### Neshyba, 2023


# A kinetic-molecular approach to reversible expansion of an ideal gas

## Introduction
This CGI is all about exploring the question, what's going on at the molecular level when we carry out an isothermal reversible expansion of a gas? Why, for example, does heat go in at all, and why is that amount of heat equal to $nRT \ ln \bigl ({V_{final} \over V_{initial}} \bigr )$? Our basic premise here is that if we think about this heat as compensatation for the loss of speed that molecules undergo when they hit a slowly receding wall, then we have achieved a degree of molecular-level insight. 


## A hybrid (numerical+theoretical) kinetic-molecular approach

We'll begin with the result of an analysis of the physics of objects slamming against a receding wall. If we can assume that that process is elastic (as physicists like to do), one can show when those objects will bounce back,  their will be speed reduced by $2v_{wall}$. That reduction in speed, in turn, implies the kinetic energy of those objects (OK, molecules) will also be reduced. If we want our process to be isothermal, then we can say that this reduction in kinetic energy would have to be compensated by an equal inflow of heat from the surroundings. 

Kinetic-molecular theory says that this inflow of heat is given by

$$
dq_{kinetic} = -Z \times \delta KE \times A \times dt \ \ \ \ (1)
$$

This is a longish expression, but we hope you can see why it works, based on the factors that go into it:

- $Z$ is the frequency with which molecules hit our wall (typical units: mole / meter$^2$ / second). Kinetic theory says $Z = {1 \over 2} \rho \times v$, where $\rho$ is the number density of the gas (the factor ${1 \over 2}$ is there because only half the molecules at any given moment are heading toward the wall in question), and $v$ is the speed of the molecules in the gas. That speed, in turn, can be inferred from the equipartition theorem, which says that each degree of freedom (in this case, motion in the x-direction) has an average energy of ${1 \over 2}RT$. We can get from kinetic energy to speed using $KE = {1 \over 2}Mv^2$, where $M$ is the mass of a mole of molecules.
- $\delta KE$ is the change in kinetic energy of a mole of molecules hitting the receding wall (typical units: J/mol). As mentioned previously, for elastic collisions, each molecule's speed will be reduced by $2v_{wall}$ after it hits the wall.
- $A$ is the area of the wall getting hit (meter$^2$)
- $dt$ is the time (seconds) needed to for the wall to move out some distance $dx$, i.e, $dx=v_{wall}dt$. This is also related to the final volume, of course.

If these considerations don't convince you that Eq. 1 makes sense, you can try one more thing: do a unit analysis. If the right hand side has dimensions energy, we're on the right track!

The notation $dq_{kinetic}$ is supposed bring to mind the idea that Eq. 1 is valid only for infintessimally small expansions -- that is, $dt$, and therefore $dx$, are small in some sense. For an expansion that's too big to be considered infinitessimally small, we'll need to add up a series of $dq_{kinetic}$ values until the gas has expanded to our final, desired volume. In calculus-y terms, we would express this idea as 

$$
q_{kinetic} = \int_{V_{init}}^{V_{final}} dq_{kinetic} \ \ \ \ (2)
$$

Simulated numerically, we'd write this as 

$$
q_{kinetic} \approx \sum_{V_{init}}^{V_{final}} dq_{kinetic} \ \ \ \ (3)
$$

which is what Numpy's np.sum function does. 

A couple of notes about Eq. (3). First, we'd need to make sure that each $dq_{kinetic}$ is small enough that the summation is a good approximation to the integral in Eq. 2. That's the sense we intended above, when we said that $dt$ and $dx$ need to be "small." Second, we'd need to make sure that the starting volume for each tiny step is the ending volume of the previous step. (We're just mentioning this because we forgot to do that the first time through this.)

## A proper kinetic-molecular approach

The foregoing has a distinct numerical/algorithmic air to it. It will give results that are as accurate as you please, as long as you make the step size built into Eq. (3) small enough. But the development also shows us how to use kinetic-molecular theory to get an analytical result. Here are some key relationships:

$$
\delta KE = {\partial KE \over \partial v} \delta v \ \ \ \ (4)
$$

where ${\partial KE \over \partial v}$ can be got from taking the derivative of $KE = {1 \over 2}Mv^2$, and $\delta v=2v_{wall}$. If you put all this together, and notice that $Adx=dV$, you can come up with

$$
dq_{kinetic,proper} =  n M v^2 {dV \over V} \ \ \ \ (5)
$$

This expression, integrated from initial to final volumes, gives

$$
q_{kinetic,proper} = n M v^2 ln \bigl ({V_{final} \over V_{initial}} \bigr ) \ \ \ \ (6)
$$

where we can justify pulling $v^2$ out of the integral because we're insisting that enough heat comes in during the process that the gas keeps the same temperature throughout the expansion.

## Regular old thermodynamic theory

And what does Thermodynamics have to say about this? Going back to a very small expansion, we can say that the infinessimal work done by an expanding gas is $dw = -P_{ex}dV$. Under isothermal and reversible conditions, for an ideal gas, and invoking the 1st Law, this translates to 
 
$$
dq_{thermo} = nRT {dV \over V} \ \ \ \ (7)
$$

For an expansion over many such steps, we can integrate over $V$, giving

$$
q_{thermo} = nRT \ ln \bigl ({V_{final} \over V_{initial}} \bigr ) \ \ \ \ (8)
$$

## The idea of this exericse
A test of the kinetic-molecular theory laid out above would, naturally, be to evaluate whether Eqs. 1, 5, and 7 give (close to) the same answers. Ditto for Eqs. 3, 6, and 8. If so, we can conclude that there's some validity to the notion that the heat that flows into an expanding gas is compensating for the reduction in the kinetic energy of molecules hitting the receding wall.

## Learning goals
The main learning goals of this exercise are 
1. I can provide a qualitative, molecular-level explanation for why heat must flow into a gas as it expands isothermally and reversibly.
1. I can use kinetic-molecular theory to predict the heat that flows into an ideal gas as it expands reversibly and isothermally.
1. I can explain the basis for that calculation, including the equipartition theory, elastic collision theory, and the frequency-of-collision theory. 
1. I can show that this formulation matches predictions using the the 1st Law of Thermodynamics, both for expansions that are tiny, and large (via integration).

In [1]:
import pint; from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Constants
R = AssignQuantity(8.314,'joule/mol/K')
NA = AssignQuantity(6.02e23,'1/mol')
T = AssignQuantity(293,'K')
n = AssignQuantity(1,'mol')

# We're assuming O2 gas here
M = AssignQuantity(32,'g/mol')

### Kinetic theory associated with isothermal processes
The average kinetic energy in the x-direction, according to the equipartition theorem, is $KE = {1 \over 2}Mv^2$, where $M$ is the mass of a mole of molecules. Use that in the cell below to evaluate the average kinetic energy of a mole of our gas molecules, and then compute the RMS speed associated with that average.

In [3]:
# Average kinetic energy in the x-direction, using the equipartition theorem
### BEGIN SOLUTION
KE = 0.5*R*T;
KE.ito('kjoule/mol')
print(KE)
### END SOLUTION

# RMS speed associated with that average
### BEGIN SOLUTION
v = (2/M*KE)**.5
v.ito('m/s')
print(v)
### END SOLUTION

1.2180010000000001 kilojoule / mole
275.90770648896347 meter / second


### Specifying the initial state of our gas
In the cell below, we specify the initial state of our gas, and the box it's in. 

In [25]:
# Specify the initial volume of our gas 
Vinit = AssignQuantity(24,'liter')
Vinit.ito('dm^3')
print('Initial volume = ', Vinit)

# Specify the starting length in the x-direction
xinit = AssignQuantity(5,'decimeter')
print('Initial distance in the x-direction = ', xinit)

# Deduce the area in the y & z directions consistent with the initial x and volume
A = Vinit/xinit
print('Sides of the wall = ', A**.5)

Initial volume =  24.000000000000007 decimeter ** 3
Initial distance in the x-direction =  5 decimeter
Sides of the wall =  2.1908902300206647 decimeter


### Hybrid kinetic-molecular theory of a tiny (infinitessimal) expansion of an ideal gas
In the cell below, we lay out some parameters of our tiny expansion -- which you can leave as-is. After that, all the calculations are meant to lead up to the calculation of $dq_{kinetic}$.

In [26]:
# Specify the distance the wall will move out (should be small, on the order of millimeters)
dx = AssignQuantity(0.1,'centimeter')
xfinal = xinit+dx
print('Final distance in the x-direction = ', xfinal)

# Also specify the wall speed -- this should be small compared to the average molecular speed (for reversibility)
wallspeed = AssignQuantity(10,'cm/s')

# We'll need the starting mole density of our gas (call it rho)
### BEGIN SOLUTION
rho = n/Vinit
rho.ito('mol/m^3')
print('Gas density = ', rho)
### END SOLUTION

# Calculate the increase in volume (dV)
### BEGIN SOLUTION
dV = dx*A
print('Increase in volume = ', dV)
### END SOLUTION

# Calculate the change in speed of a molecule hitting the wall (delta_v)
### BEGIN SOLUTION
delta_v = 2*wallspeed
print('Change in speed = ', delta_v)
### END SOLUTION

# Calculate the collision frequency (Z) assuming half the molecules are moving toward the receding wall
### BEGIN SOLUTION
Z = rho/2*v
print('Collision frequency = ', Z)
### END SOLUTION

# Calculate the resulting change in KE per mole of molecules hitting the wall (delta_KE)
### BEGIN SOLUTION
delta_KE = .5*M*((v-delta_v)**2-v**2)
delta_KE.ito('kilojoule/mol')
print('Change in KE per mole of collisions =', delta_KE)
### END SOLUTION

# Calculate how long it takes to move that distance (dt)
### BEGIN SOLUTION
dt = dx/wallspeed
dt.ito('sec')
print('Infinitessimal time needed to expand out a bit = ', dt)
### END SOLUTION

# Calculate the heat required to compensate for the kinetic energy that is lost (dq_kinetic)
### BEGIN SOLUTION
dq_kinetic = -delta_KE*Z*A*dt
dq_kinetic.ito('joule')
print(dq_kinetic)
### END SOLUTION

Final distance in the x-direction =  5.01 decimeter
Gas density =  41.66666666666665 mole / meter ** 3
Increase in volume =  0.4800000000000002 centimeter * decimeter ** 2
Change in speed =  20 centimeter / second
Collision frequency =  5748.07721852007 mole / meter ** 2 / second
Change in KE per mole of collisions = -0.0017651693215293344 kilojoule / mole
Infinitessimal time needed to expand out a bit =  0.01 second
4.870238190678385 joule


### Thermodynamics of a tiny (infinitessimal) expansion
Below, calculate the infinitessimal amount of heat that has to go in to maintain isothermal conditions for reversible expansion of an ideal gas by a tiny amount ($dV$), according to thermoynamics (Eq. 7).

In [6]:
# The infinitessimal dq according to thermo (dq_thermo) -- see Eq. 7
### BEGIN SOLUTION
dq_thermo = n*R*T*dV/Vinit
dq_thermo.ito('joule')
print(dq_thermo)
### END SOLUTION

4.8720040000000004 joule


### Pause for analysis
Calculate (and print) the difference (%) between the infinitessimal heat amount obtained from kinetic theory, and your thermodynamic result. 

In [7]:
### BEGIN SOLUTION
error = (dq_kinetic-dq_thermo)/dq_thermo*100
print('Error = ', error)
### END SOLUTION

Error =  -0.03624400393792248 dimensionless


### $dq_{kinetic,proper}$
Thinking ahead to our plans to integrate our results over larger expansions than just the infinitessimal, it'll be handy to have a "functionalized" way of computing the infinitessimal heating for a small expansion. We do that in the cell below, making use of Eq. 5 in the Introduction.

The second part of this cell makes a call to this function to make sure it works.

In [8]:
def get_dq_kinetic_proper(n, M, v, V, dx, A):
    ###  This is a functionalized computation of the infinitessimal heating for a small expansion. ###
    
    dV = A*dx
    dq_kinetic = n*M*v**2*dV/V
    dq_kinetic.ito('joule')
    
    return dq_kinetic

# Testing the algorithm -- hopefully we get the same dq as what we got before
dq_kinetic_proper = get_dq_kinetic_proper(n, M, v, Vinit, dx, A)
print('dq (test) = ', dq_kinetic_proper)

dq (test) =  4.872004000000001 joule


### Pause for analysis
In the cell below, comment on the result you just got: does $dq_{kinetic,proper}$ agree (more or less) with $dq_{kinetic}$ and $dq_{thermo}$? 

### BEGIN SOLUTION

Yes

### END SOLUTION

### Numerical integration of our kinetic-molecular results
Assuming we're happy with our get_dq_kinetic_proper function, we'll put it in a loop to simulate many steps -- see Eq. 3 in the Introduction.

In [23]:
# Specify how much the wall will move out (can be big because we're going to integrate!)
deltax = AssignQuantity(100,'centimeter')
print('Moving the wall from ', xinit, ' to ', xinit+deltax) 

# Also need to specify the number of steps, and the dx associated with each step
nsteps = 50
dx = deltax/nsteps

# Loop over our steps, updating our starting volume and recording the heat as we go along
dq_kinetic = AssignQuantity([],'joule')
for i in range(nsteps):
    
    # This finds the volume at this point in the expansion
    Vinit_this_step = Vinit + i*dx*A
    
    # Make a call to get_dq_kinetic_proper to get dq_kinetic_this_step
    ### BEGIN SOLUTION
    dq_kinetic_this_step = get_dq_kinetic_proper(n, M, v, Vinit_this_step, dx, A)
    ### END SOLUTION
    
    # This appends the heat transfer for this step to an ever-growing list of dq's
    dq_kinetic = np.append(dq_kinetic, dq_kinetic_this_step)

# This is the numerical equivalent of integrating over all the steps 
print("Here's our long list of dq values:")
print(dq_kinetic,'\n')

# This is the numerical equivalent of integrating over all the steps (Eq. 3)
q_kinetic = np.sum(dq_kinetic)
print('q_kinetic from numerical integration = ', q_kinetic)

Moving the wall from  5 decimeter  to  15.0 decimeter
Here's our long list of dq values:
[97.44008000000002 93.69238461538463 90.2222962962963 87.00007142857143 84.00006896551726 81.20006666666669 78.58070967741938 76.12506250000001 73.81824242424244 71.64711764705882 69.60005714285717 67.66672222222223 65.83789189189191 64.1053157894737 62.461589743589755 60.900050000000014 59.41468292682928 58.000047619047635 56.65120930232558 55.36368181818184 54.13337777777778 52.95656521739131 51.82982978723405 50.750041666666675 49.71432653061225 48.72004000000001 47.76474509803923 46.84619230769231 45.96230188679246 45.11114814814815 44.29094545454546 43.500035714285715 42.73687719298246 42.00003448275863 41.28816949152543 40.600033333333336 39.93445901639345 39.29035483870969 38.66669841269842 38.06253125 37.476953846153854 36.90912121212122 36.35823880597015 35.82355882352941 35.30437681159421 34.800028571428584 34.30988732394367 33.83336111111112 33.36989041095891 32.91894594594596] joule 

q

### Analytical integration of our kinetic-molecular results 
In the cell below, we propose a final volume that's a lot bigger than the initial volume. Then we evaluate the heat transferred using our integrated kinetic-molecular formula, Eq. 6.

In [21]:
Vfinal = Vinit + deltax*A
print('Initial and final volumes = ', Vinit, Vfinal)
q_kinetic_proper = n*M*v**2*np.log(Vfinal/Vinit)
q_kinetic_proper.ito('joule')
print('q_kinetic from analytical integration = ', q_kinetic_proper)

Initial and final volumes =  24.000000000000007 decimeter ** 3 72.00000000000003 decimeter ** 3
q_kinetic from analytical integration =  2676.2217324200933 joule


### Integrating our results -- thermodynamic theory
See Eq. 8.

In [24]:
# According to thermo (Eq. 8)
### BEGIN SOLUTION
q_thermodynamic = n*R*T*np.log(Vfinal/Vinit)
print(q_thermodynamic)

# Error
error_numerical = (q_kinetic/q_thermodynamic-1)*100
print('Error of the numerical result (%) = ', error_numerical)

error_analytical = (q_kinetic_proper/q_thermodynamic-1)*100
print('Error of the analytical result (%) = ', error_analytical)

### END SOLUTION

2676.221732420093 joule
Error of the numerical result (%) =  1.2244384062375158 dimensionless
Error of the analytical result (%) =  2.220446049250313e-14 dimensionless


### Pause for analysis
How close (%) did your numerical result from kinetic theory come to your thermodynamic result? What do you suppose would be a good strategy for improving that agreement?

How close did your proper analytical result come to your integrated thermodynamic result? Is the agreement satisfactory, in your view?

By the way, in situations like this, if an error comes in at, say, $10^{-10} \%$ or smaller we can assume that error is due to numerical issues (like how precise Python stores floating point numbers), so it's really zero.

### BEGIN SOLUTION

Agreement with the numerical result is around 1%. It could be improved by more steps in the integration loop.  
Agreement with the analytical result is 10^-14, which is basically zero.

### END SOLUTION

### Refreshing and saving your code
1. Use the dropdown menu Kernel/Restart
2. Use the dropdown menu Cell/Run All Above
3. Under the "File" dropdown menu item in the upper left is a disk icon. Press it now to save your work (you can, do this at any time as you're working on an assignment, actually).

### Validating
This step will help ensure that you didn't miss something (although it's not a guarantee). Find the "Validate" button and press it. If there are any errors or warnings, fix them.

### Finishing up
Assuming all this has gone smoothly, carry out three more steps (but read this carefully before starting):
1. Close this notebook using the "File/Close and Halt" dropdown menu
1. Using the Assignments tab, submit this notebook
1. Press the Logout tab of the Home Page